In [ ]:
import ipfshttpclient as ip
import os
cli = ip.connect()
print(cli.id()['ID'])

In [ ]:
res = cli.add('Zdumx') # STORE THING INSIDE IPFS
hh = res['Hash'] # GET HASH
# cli.cat(hh) # get the textual stuff | CANNOT RETURN

In [ ]:
location = './test' # this is a FOLDER
cli.get(hh, location) # put the hash into that folder

In [ ]:
os.remove('./test/'+hh) # DELETE LATER
cli.close() # CLOSE

In [ ]:
a = 3, 7
b = 4, 8

c = []
c.append(a)
c.append(b)

for k, f in c:
    print(k, f)

In [ ]:
#EMPTY MODEL FOLDER
ls = os.listdir('./models')
for x in ls:
    os.remove('./models/' + x)

In [ ]:

class A:
    def __init__(self, p):
        self.x = 1
        self.p = [i for i in range(p)]
class B:
    def __init__(self, a):
        self.a = a
        self.a.x = 2
class C:
    def __init__(self):
        self.a = None
    def aa(self, a):
        self.a = a
    def b(self):
        self.a.p.append(3)

a = A(4)
print("Original: ", a.x, a.p)
b = B(a)
print("After B : ", a.x, a.p)
c = C()
c.aa(a)
c.a.x += 4
c.b()
print("After C : ", a.x, a.p)

In [ ]:
class X:
    def __init__(self):
        pass

f = X()
g = X()
h = X()

j = [f,g]
k = [f,g,h]

for n in k:
    if n not in j:
        print(str(n) + "not in k")

In [ ]:
from copy import deepcopy
a = {} #prev
b = {'1': 2, '2': 3}
k = {'1': 1, '2': 4}

def update(old,new):
    upd = {}
    for t, q in new.items():
        if t not in old.keys():
            upd[t] = q
        elif upd[t] < q:
            upd[t] = q
    print(upd)
            

print(update(b, k))

In [ ]:
a = "a;b;c;"
v = a.split(';')
v


In [ ]:
import json
j = {'a':1,  'b':2}
j = str(j)
k = json.loads(j.replace("\'","\""))

In [ ]:
a = []
b = [1,3,5]
a.append(b)

In [ ]:
a += b
a

In [2]:
# deeper cnn model for mnist
from numpy import mean
from numpy import std
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# define model
		model = define_model()
		# select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
		# evaluate model
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# stores scores
		scores.append(acc)
		histories.append(history)
	return scores, histories

# plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		plt.subplot(2, 1, 1)
		plt.title('Cross Entropy Loss')
		plt.plot(histories[i].history['loss'], color='blue', label='train')
		plt.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		plt.subplot(2, 1, 2)
		plt.title('Classification Accuracy')
		plt.plot(histories[i].history['accuracy'], color='blue', label='train')
		plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	plt.show()

# summarize model performance
def summarize_performance(scores):
	# print summary
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
	# box and whisker plots of results
	plt.boxplot(scores)
	plt.show()

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# evaluate model
	scores, histories = evaluate_model(trainX, trainY)
	# learning curves
	summarize_diagnostics(histories)
	# summarize estimated performance
	summarize_performance(scores)

# entry point, run the test harness
run_test_harness()

> 98.917
> 99.033


In [ ]:
model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
model.save('final_model.h5')

In [6]:
import pandas as pd
import os

p = []
for subdir, dirs, files in os.walk("./dataset/tser"):
    for filename in files:
        filepath = subdir + '/' + filename
        p.append(filepath)

for j in p:
    k = pd.read_csv(j)
    k = k.drop(columns=['start','end'])
    k.to_csv(j[:-4]+'2.csv')

In [3]:
a = 24,
print(len(a))

1
